<a href="https://colab.research.google.com/github/kammer0820/gym-super-mario-bros/blob/master/%E6%BC%94%E7%BF%923_mario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ライブラリのインストール**

In [40]:
!pip install gym-super-mario-bros > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install torch torchvision > /dev/null 2>&1
!pip install scikit-learn > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)


# **ライブラリのインポート**

In [0]:
#マリオ関連のimport
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym_super_mario_bros.actions import RIGHT_ONLY


#プロット関連のimport
import matplotlib.pyplot as plt
from matplotlib import animation, rc

#vcopt関連のimport
import numpy as np
import numpy.random as nr

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
import torch
from IPython.display import HTML

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

import time
import random
import numpy as np
from collections import deque
import tensorflow as tf
from matplotlib import pyplot as plt

In [42]:
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1011'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1011'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video(command_list):
  env = wrap_env(gym_super_mario_bros.make('SuperMarioBros-1-1-v0'))
  env = JoypadSpace(env, SIMPLE_MOVEMENT)

  env.reset()
  for i in range(len(command_list)):
      env.render()
      env.step(command_list[i])
  env.close()

  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
      mp4 = mp4list[0]
      video = io.open(mp4, 'r+b').read()
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                  loop controls style="height: 400px;">
                  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# **マリオでテスト**

In [0]:
class QNet(torch.nn.Module):
    def __init__(self,in_dim,out_dim,n_hid):
        super(QNet, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.n_hid = n_hid
        
        self.fc1 = torch.nn.Linear(in_dim,n_hid,'relu')
        self.fc2 = torch.nn.Linear(n_hid,out_dim,'linear')
        
    def forward(self,x):
        print("forwardしてます")
        x = F.relu(self.fc1(x))
        y = self.fc2(x)
        return y

class DQNAgent:
    """ DQN agent """
    def __init__(self, states, actions, max_memory, double_q):
        self.states = states
        self.actions = actions
        self.model = QNet(self.states ,actions, 64)
        self.target_model = copy.deepcopy(self.model)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=0.001)
        #self.session.run(tf.global_variables_initializer())
        self.memory = deque(maxlen=max_memory)
        self.eps = 1
        self.eps_decay = 0.99999975
        self.eps_min = 0.1
        self.gamma = 0.90
        self.batch_size = 32
        self.burnin = 100000
        self.copy = 10000
        self.step = 0
        self.learn_each = 3
        self.learn_step = 0
        self.save_each = 500000
        self.double_q = double_q

    def copy_model(self):   #copy weights to target network
        self.target_model.load_state_dict(self.model.state_dict())
    def save_model(self):
        """ Saves current model to disk """
        torch.save(self.model.state_dict())
    def add(self, experience):
        """ Add observation to experience """
        self.memory.append(experience)
    def predict(self, model, state):
        """ Prediction """
        if model == 'online':
            return self.model(state)
        if model == 'target':
            return self.target_model(state)
    def run(self, state):
        """ Perform action """
        if np.random.rand() < self.eps:    #random action
            action = np.random.randint(low=0, high=self.actions)
        else:                         # Policy action
            q = self.predict('online', np.expand_dims(state, 0))
            action = np.argmax(q)
        # Decrease eps
        self.eps *= self.eps_decay
        self.eps = max(self.eps_min, self.eps)
        # Increment step
        self.step += 1
        return action
    def learn(self):
        """ Gradient descent """
        # Sync target network
        if self.step % self.copy == 0:
            self.copy_model()
        # Checkpoint model
        if self.step % self.save_each == 0:
            self.save_model()
        # Break if burn-in
        if self.step < self.burnin:
            return
        # Break if no training
        if self.learn_step < self.learn_each:
            self.learn_step += 1
            return
        # Sample batch
        batch = random.sample(self.memory, self.batch_size)
        state, next_state, action, reward, done = map(np.array, zip(*batch))
        # Get next q values from target network
        next_q = self.predict('target', next_state)
        # Calculate discounted future reward
        if self.double_q:
            q = self.predict('online', next_state)
            a = np.argmax(q, axis=1)
            target_q = reward + (1. - done) * self.gamma * next_q[np.arange(0, self.batch_size), a]
        else:
            target_q = reward + (1. - done) * self.gamma * np.amax(next_q, axis=1)
        # Update model
        summary, _ = self.session.run(fetches=[self.summaries, self.train],
                                      feed_dict={self.input: state,
                                                self.q_true: np.array(target_q),
                                                self.a_true: np.array(action),
                                                self.reward: np.mean(reward)})
        # Reset learn step
        self.learn_step = 0
        # Write
        self.writer.add_summary(summary, self.step)


In [45]:
#環境の用意
env = wrap_env(gym_super_mario_bros.make('SuperMarioBros-1-1-v0'))
env = JoypadSpace(env, RIGHT_ONLY)

# Parameters
states = (84, 84, 4)
actions = env.action_space.n

# Agent
agent = DQNAgent(states=states, actions=actions, max_memory=100000, double_q=True)

#パラメータ
n_episodes = 3
episode_count = 0

#記録用
action_history = []
rewards = []


# Timing
start = time.time()
step = 0

#繰り返し操作して画面を表示
while(episode_count < n_episodes):
    
    #ゲーム環境のリセット
    state = env.reset()
    action_res = []

    total_reward = 0
    step_count = 0

    while(True):
      action = agent.run(state=state)
      next_state, reward, done, info = env.step(action)
      agent.add(experience=(state, next_state, action, reward, done))
      agent.learn()

      total_reward += reward

      state = next_state

      action_res.append(action)
      step_count += 1

      if done or info['flag_get']:
          episode_count += 1
          break

    rewards.append(total_reward / step_count)

     # Print
    if episode_count % 100 == 0:
        print('Episode {e} - +'
              'Frame {f} - +'
              'Frames/sec {fs} - +'
              'Epsilon {eps} - +'
              'Mean Reward {r}'.format(e=e,
                                       f=agent.step,
                                       fs=np.round((agent.step - step) / (time.time() - start)),
                                       eps=np.round(agent.eps, 4),
                                       r=np.mean(rewards[-100:])))
        start = time.time()
        step = agent.step
        
    print(episode_count, "done")
    action_history.append(action_res)

env.close()
# Save rewards
np.save('rewards.npy', rewards)

AttributeError: ignored

# **ビデオの表示**

In [29]:
show_video(action_history[0])

/usr/local/lib/python3.6/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Exception ignored in: <bound method Monitor.__del__ of <Monitor<TimeLimit<SuperMarioBrosEnv<SuperMarioBros-1-1-v0>>>>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/gym/wrappers/monitor.py", line 229, in __del__
    self.close()
  File "/usr/local/lib/python3.6/dist-packages/gym/wrappers/monitor.py", line 134, in close
    super(Monitor, self).close()
  File "/usr/local/lib/python3.6/dist-packages/gym/core.py", line 238, in close
    return self.env.close()
  File "/usr/local/lib/python3.6/dist-packages/gym/core.py", line 238, in close
    return self.env.close()
  File "/usr/local/lib/python3.6/dist-packages/nes_py/nes_env.py", line 338, in close
    raise ValueError('env has already been closed.')
ValueError: env has already been closed.
